In [1]:
import json
def dump_jsonl(data, output_path, append=False):
    """
    Write list of objects to a JSON lines file.
    """
    mode = 'a+' if append else 'w'
    with open(output_path, mode, encoding='utf-8') as f:
        for line in data:
            json_record = json.dumps(line, ensure_ascii=False)
            f.write(json_record + '\n')
    print('Wrote {} records to {}'.format(len(data), output_path))

def load_jsonl(input_path) -> list:
    """
    Read list of objects from a JSON lines file.
    """
    data = []
    with open(input_path, 'r', encoding='utf-8') as f:
        for line in f:
            if line != '\n':
                data.append(json.loads(line.rstrip('\n')))
    print('Loaded {} records from {}'.format(len(data), input_path))
    return data

In [2]:
fname = r"C:\Users\gbao\Google 云端硬盘\BUDT758X\Final project\stream__realDonaldTrump.jsonl"
tweet_data = load_jsonl(fname)

Loaded 138859 records from C:\Users\gbao\Google 云端硬盘\BUDT758X\Final project\stream__realDonaldTrump.jsonl


In [3]:
import pandas as pd
db_data = []
# first level keys
l1_cols = ['id','created_at','text','source','geo','coordinates',
           'is_quote_status','quoted_status','quoted_status_id',
           'in_reply_to_status_id','in_reply_to_user_id',
           'quote_count','reply_count','retweet_count','favorite_count']
# keys under user
us_cols = ['id','screen_name','created_at','friends_count','follow_count','verified']
# keys under entities
en_cols = ['hashtags']
ex_cols = ['full_text']
for d in tweet_data:
    db_data.append([])
    for i in l1_cols:
        db_data[-1].append(d.get(i, float('nan')))
    temp = d.get('user',True)
    for i in us_cols:
        db_data[-1].append(temp.get(i, float('nan')))
    temp = d.get('entities',float('nan'))
    for i in en_cols:
        db_data[-1].append(temp.get(i, float('nan')))
    temp = d.get( 'extended_tweet',False)
    if temp:
        for i in ex_cols:
            db_data[-1].append(temp.get('full_text', float('nan')))
columns = l1_cols+us_cols+en_cols+ex_cols
df = pd.DataFrame(db_data, columns=columns)

In [4]:
df.head()

,id,created_at,text,source,geo,coordinates,is_quote_status,quoted_status,quoted_status_id,in_reply_to_status_id,...,retweet_count,favorite_count,id,screen_name,created_at,friends_count,follow_count,verified,hashtags,full_text
0,1255068528849215489,Tue Apr 28 09:37:09 +0000 2020,@realDonaldTrump why should the American peopl...,"<a href=""http://twitter.com/download/iphone"" r...",None,None,False,NaN,NaN,1.254783e+18,...,0,0,938787819580923904,505abq503pdx,Thu Dec 07 15:10:32 +0000 2017,115,NaN,False,[],@realDonaldTrump why should the American peopl...
1,1255068533555212288,Tue Apr 28 09:37:11 +0000 2020,@realDonaldTrump @GOP @SpeakerPelosi \n@FoxNe...,"<a href=""http://twitter.com/download/android"" ...",None,None,False,NaN,NaN,NaN,...,0,0,1206767177841295360,Levelhead14,Tue Dec 17 02:45:08 +0000 2019,97,NaN,False,[],@realDonaldTrump @GOP @SpeakerPelosi \n@FoxNe...
2,1255068535551737859,Tue Apr 28 09:37:11 +0000 2020,RT @realDonaldTrump: Why should the people and...,"<a href=""http://twitter.com/download/android"" ...",None,None,False,NaN,NaN,NaN,...,0,0,1243960919618498562,pYx0eiPwAJeQpWF,Sat Mar 28 17:59:49 +0000 2020,23,NaN,False,[],None
3,1255068537372246019,Tue Apr 28 09:37:12 +0000 2020,"RT @realDonaldTrump: FAKE NEWS, THE ENEMY OF T...","<a href=""http://twitter.com/download/iphone"" r...",None,None,False,NaN,NaN,NaN,...,0,0,553805586,MassiveMatthew,Sat Apr 14 18:04:33 +0000 2012,1416,NaN,False,[],None
4,1255068537850335232,Tue Apr 28 09:37:12 +0000 2020,@realDonaldTrump Old dirty Donnie will be taki...,"<a href=""http://twitter.com/download/android"" ...",None,None,False,NaN,NaN,1.254967e+18,...,0,0,1219699627806613504,Timothy70078813,Tue Jan 21 19:15:07 +0000 2020,1,NaN,False,[],None


In [14]:
import json
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import tokenizer_from_json
from keras.preprocessing.sequence import pad_sequences

with open('tokenizer.json', 'r') as f:
    jstring = json.load(f)
tokenizer = tokenizer_from_json(jstring)
model = keras.models.load_model('sentiment_model.h5')


In [15]:
import re
import nltk
from nltk.corpus import stopwords
from  nltk.stem import SnowballStemmer
stop_words = stopwords.words("english")
stemmer = SnowballStemmer("english")

def preprocess(text, stem=False):
    # Remove link,user and special characters
    text = re.sub("@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+", ' ', str(text).lower()).strip()
    tokens = []
    for token in text.split():
        if token not in stop_words:
            if stem:
                tokens.append(stemmer.stem(token))
            else:
                tokens.append(token)
    return " ".join(tokens)

x = df.full_text.dropna(how='any').apply(lambda x: preprocess(x))
x = pad_sequences(tokenizer.texts_to_sequences(x), maxlen=300)

In [16]:
SENTIMENT_THRESHOLDS = (0.4, 0.7)
def decode_sentiment(score, include_neutral=True):
    if include_neutral:        
        label = 'NEUTRAL'
        if score <= SENTIMENT_THRESHOLDS[0]:
            label = 'NEGATIVE'
        elif score >= SENTIMENT_THRESHOLDS[1]:
            label =' POSITIVE'

        return label
    else:
        return 'NEGATIVE' if score < 0.5 else 'POSITIVE'
    
import time
def predict(text, include_neutral=True):
    start_at = time.time()
    # Tokenize text
    x_test = pad_sequences(tokenizer.texts_to_sequences([text]), maxlen=300)
    # Predict
    score = model.predict([x_test])[0]
    # Decode sentiment
    label = decode_sentiment(score, include_neutral=include_neutral)

    return {"label": label, "score": float(score),
       "elapsed_time": time.time()-start_at}  

In [17]:
scores = model.predict(x, verbose=1, batch_size=8000)
label = [decode_sentiment(score, include_neutral=False) for score in scores]

22999/22999 [==============================] - 4s 173us/sample
